In [1]:
%matplotlib qt

from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
import cv2
from skimage import data, util
from skimage.measure import label, regionprops
import numpy as np
import math
import os

In [2]:
def muCalculator(pointsArray):
    mu = np.mean(pointsArray, axis=0)
    return mu

def sigmaCovCalculator(pointsArray):
    mu = muCalculator(pointsArray)
    sigma = (1/pointsArray.shape[0]) * np.dot((pointsArray - mu).T , (pointsArray - mu))
    return sigma

def gaussian(pointsArray, mu, sigma):
    gaussian = np.zeros((pointsArray.shape[0],1))
    for pointIndex, point in enumerate(pointsArray):
        difference = (point-mu).T
        multiplier = 1/ ( math.pow((2*math.pi),(3/2)) * (math.pow( np.linalg.det(sigma) , (1/2) ) ) )
        exponent = math.exp(-0.5 * np.dot(np.dot( difference.T, np.linalg.inv(sigma)), difference ))
        output = multiplier * exponent
        gaussian[pointIndex] = output
    return gaussian

def calculateGMMProbabilities(inputImage, clusters):
    GMMOutput = np.zeros((inputImage.shape[0], 1))
    for cluster in clusters:
        simpleGaussianOutput = gaussian(inputImage, cluster["mu"], cluster["sigma"])
        GMMOutput = GMMOutput + ((1/clusters.shape[0]) * simpleGaussianOutput)
    return GMMOutput

In [3]:
clusters = np.load('clusters.npy', allow_pickle=True)

### Image 001

In [13]:
print("Calculating probabilities for file: 001.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/001.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/001.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 5)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 001" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 001.png...
Conducting image filtering and computations...
Test Image Number: 001,  Centroid Row:  641.4728544008774,  Centroid Column:  476.5248423361667,  Distance: 2.58m


### Image 002

In [11]:
print("Calculating probabilities for file: 002.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/002.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/002.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(50,50))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 1)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 55)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 002" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 002.png...
Conducting image filtering and computations...


### Image 003

In [17]:
print("Calculating probabilities for file: 003.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/003.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/003.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 5)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 003" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 003.png...
Conducting image filtering and computations...
Test Image Number: 003,  Centroid Row:  552.8221249502586,  Centroid Column:  386.0112415439713,  Distance: 4.92m
Test Image Number: 003,  Centroid Row:  698.2749711268767,  Centroid Column:  449.70678105923116,  Distance: 2.83m


### Image 004

In [21]:
print("Calculating probabilities for file: 004.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/004.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/004.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 9)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 004" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 004.png...
Conducting image filtering and computations...
Test Image Number: 004,  Centroid Row:  632.9548271675932,  Centroid Column:  445.67045769173427,  Distance: 2.28m


### Image 005

In [24]:
print("Calculating probabilities for file: 005.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/005.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/005.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 10)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 5)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1000:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 005" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 005.png...
Conducting image filtering and computations...
Test Image Number: 005,  Centroid Row:  644.7899686520376,  Centroid Column:  482.00313479623827,  Distance: 9.76m


### Image 006

In [32]:
print("Calculating probabilities for file: 006.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/006.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/006.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 7)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 7)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 006" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 006.png...
Conducting image filtering and computations...
Test Image Number: 006,  Centroid Row:  157.71558878657413,  Centroid Column:  584.7173865302016,  Distance: 1.11m
Test Image Number: 006,  Centroid Row:  42.933853459972866,  Centroid Column:  389.88297150610583,  Distance: 9.08m
Test Image Number: 006,  Centroid Row:  632.1758436944938,  Centroid Column:  427.85168738898756,  Distance: 10.39m
Test Image Number: 006,  Centroid Row:  483.98002365619664,  Centroid Column:  835.0775397555526,  Distance: 5.65m


### Image 007

In [33]:
print("Calculating probabilities for file: 007.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/007.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/007.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 1)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 5)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 007" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 007.png...
Conducting image filtering and computations...
Test Image Number: 007,  Centroid Row:  565.6879045307444,  Centroid Column:  629.3069377022654,  Distance: 2.86m


### Image 008

In [36]:
print("Calculating probabilities for file: 008.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/008.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/008.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 2)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 008" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 008.png...
Conducting image filtering and computations...
Test Image Number: 008,  Centroid Row:  737.3641777370591,  Centroid Column:  478.0746678882272,  Distance: 10.55m


### Image 009

In [37]:
print("Calculating probabilities for file: 009.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/009.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/009.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 2)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 009" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 009.png...
Conducting image filtering and computations...
Test Image Number: 009,  Centroid Row:  673.2035590277778,  Centroid Column:  428.07769097222223,  Distance: 10.27m


### Image 010

In [42]:
print("Calculating probabilities for file: 010.png...")
# Reading images as RGB and HSV
img_RGB = cv2.cvtColor( cv2.imread("Test_Set/010.png", 1) , cv2.COLOR_BGR2RGB)
img_HSV = cv2.cvtColor( cv2.imread("Test_Set/010.png", 1) , cv2.COLOR_BGR2HSV)
img_dim = img_RGB.shape[0]*img_RGB.shape[1]

# make sure the input image is reshaped and normalized
input_image = img_HSV.reshape(img_dim, 3)
input_image = input_image[input_image.sum(axis=(1)) != 0]/255

# do the same to RGB version for plotting later
input_image_RGB = img_RGB.reshape(img_dim, 3)
input_image_RGB = input_image_RGB[input_image_RGB.sum(axis=(1)) != 0]/255

# calculate GMM probabilities
GMMOutput = calculateGMMProbabilities(input_image, clusters)


print("Conducting image filtering and computations...")
# image erosion and dilation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
GMMOutput = np.float64(GMMOutput)
GMMOutput = cv2.erode(GMMOutput, kernel, iterations = 6)
GMMOutput = cv2.dilate(GMMOutput, kernel, iterations = 3)

# replace image with binary (1,0) for white and black
GMMOutput = np.where( GMMOutput > 0.99, 1, 0)

# label image with skimage
label_img = label(GMMOutput.reshape(img_RGB.shape[0], img_RGB.shape[1]), connectivity=2)

# display results
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(input_image_RGB.reshape(img_RGB.shape[0], img_RGB.shape[1], 3))
for region in regionprops(label_img):
    if region.area >= 1200:
        bboxAspect = (region.bbox[2]-region.bbox[0])/(region.bbox[3]-region.bbox[1])
        if (bboxAspect > 1.0325 and bboxAspect < 1.9528):
            minr, minc, maxr, maxc = region.bbox
            rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr, fill=False, edgecolor='green', linewidth=2)
            ax.add_patch(rect)
            ratio = 493.0884217183345
            barrelDistance = str( round(ratio/ math.sqrt(region.area) , 2)) + 'm'
            plt.annotate(barrelDistance, (region.centroid[1], region.centroid[0]), color='white')
            plt.show()
            print("Test Image Number: 010" + ",  Centroid Row:  " + str(region.centroid[1]) + ",  Centroid Column:  " + str(region.centroid[0]) + ",  Distance: " + str(barrelDistance))

Calculating probabilities for file: 010.png...
Conducting image filtering and computations...
Test Image Number: 010,  Centroid Row:  1034.539039039039,  Centroid Column:  273.70007507507506,  Distance: 9.55m
Test Image Number: 010,  Centroid Row:  664.2349540078843,  Centroid Column:  396.9540078843627,  Distance: 7.99m
